# Preprocessing data to Estimate the heating energy model

* Collecting the hot water data to compute the heating energy output
* Input to the model: preheat temp/st pt au 1&2, reheat temp/st pt ahu 1&2, zone temps, outside temp and rh

In [1]:
#import modules
from pandas import *
from matplotlib import *
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
%matplotlib notebook

# Collecting data for hot water energy calculation

In [ ]:
#Reading the data to calculate the energy
df3 = df3 = read_excel('TotalEnergy2018.xlsx',sheet_name='Sheet3')
#sample type 1 date='11/4/18 1:00:00 AM CDT'
#sample type 2 date='11/4/18 2:00:00 AM CST'
df3.Time = df3.Time.map(lambda x: x.rsplit(None, 1)[0])#removing time zone information #no axis info since it is a series
df3.insert(loc=0, column='Dates', value=to_datetime(df3['Time'],format='%m/%d/%y, %I:%M:%S %p'))

#Converting to Central Standard Time
CDTStart = 19896
CDTEnd = 110113+1#Since indexing ignores last value
df3['Dates'].iloc[CDTStart:CDTEnd] = df3['Dates'].iloc[CDTStart:CDTEnd] - DateOffset(hours=1)

#Setting Dates as the dataframe index
df3.set_index(['Dates'], drop=True, inplace=True)
df3.drop('Time',axis=1,inplace=True)#Drop original Time column

#Selecting time points only at 5 minute intervals
mask = np.logical_and((df3.index.minute % 5) == 0, (df3.index.second) == 0)
df3 = df3[mask]

#Dropping any row with ill conditioned data like ??? etc
for i in df3.columns:
    df3 = df3[~df3[i].apply(lambda x: isinstance(x, str))]
    
#Ensuring objects are all converted to float
df3 = df3.astype('float')

#Dropiing duplicated time points that may exist in the data
df3 = df3[~df3.index.duplicated()]
df3.index.is_unique #Statutory Check

#Dropping rows with values < 0
for i in df3.columns:
    df3 = df3[df3[i]>0]
    
#Dropping any empty row
df3.dropna(axis=0,how='any',inplace=True)

### Calculating the enrgy based on the dataframe
Used the formula
$H = 500*(FlowRate)*(SupplyTemp-ReturnTemp)$

In [ ]:
def HeatEnergy(x):
    tempDiff = x['Hot Water Supply Temperature.Hot Water Supply Temperature.Trend - Present Value ()']-x['Hot Water Return Temperature.Hot Water Return Temperature.Trend - Present Value ()']
    energy = 500*x['Flow Rate.Trend - Present Value']*tempDiff
    return energy

df3 = df3[df3['Flow Rate.Trend - Present Value']>0]
df3['Heat_energy'] = df3.apply(lambda x: HeatEnergy(x) , axis=1)

# OAT, DAT, DAT_STP for AHU1

In [ ]:
#Reading the outside air temperature, da temp and st point for ahu 1
df2 = read_excel('TotalEnergy2018.xlsx',sheet_name='Sheet2')
#sample type 1 date='11/4/18 1:00:00 AM CDT'
#sample type 2 date='11/4/18 2:00:00 AM CST'
df2.Time = df2.Time.map(lambda x: x.rsplit(None, 1)[0])#removing time zone information #no axis info since it is a series
df2.insert(loc=0, column='Dates', value=to_datetime(df2['Time'],format='%m/%d/%y, %I:%M:%S %p'))

#Converting to Central Standard Time
CDTStart = 19896
CDTEnd = 87918+1#Since indexing ignores last value
df2['Dates'].iloc[CDTStart:CDTEnd] = df2['Dates'].iloc[CDTStart:CDTEnd] - DateOffset(hours=1)

#Setting Dates as the dataframe index
df2.set_index(['Dates'], drop=True, inplace=True)
df2.drop('Time',axis=1,inplace=True)#Drop original Time column

#Selecting time points only at 5 minute intervals
mask = np.logical_and((df2.index.minute % 5) == 0, (df2.index.second) == 0)
df2 = df2[mask]

#Dropping any row with ill conditioned data like ??? etc
for i in df2.columns:
    df2 = df2[~df2[i].apply(lambda x: isinstance(x, str))]

#Ensuring objects are all converted to float
df2 = df2.astype('float')

#Dropiing duplicated time points that may exist in the data
df2 = df2[~df2.index.duplicated()]
df2.index.is_unique #Statutory Check

#Dropping rows with values < 0
for i in df2.columns:
    df2 = df2[df2[i]>0]

#Dropping empty rows
df2.dropna(axis=0,how='any',inplace=True)

# Relative Humidity

In [ ]:
#Reading the different Excel Sheets containing different Data Columns
#Reading the Relative humidity data
df4 = read_excel('BdxData.xlsx',sheet_name='Sheet1')
df4.insert(loc=0, column='Dates', value=to_datetime(df4['Date'],format='%m/%d/%Y %H:%M'))

#Setting Dates as the dataframe index
df4.set_index(['Dates'], drop=True, inplace=True)
df4.drop('Date',axis=1,inplace=True)#Drop original Time column

#Dropping any row with ill conditioned data like ??? etc
for i in df4.columns:
    df4 = df4[~df4[i].apply(lambda x: isinstance(x, str))]

#Ensuring objects are all converted to float
df4 = df4.astype('float')

#Dropiing duplicated time points that may exist in the data
df4 = df4[~df4.index.duplicated()]
df4.index.is_unique #Statutory Check

#Dropping empty rows
df4.dropna(axis=0,how='any',inplace=True)

# AHU2 DAT DAT_STP

In [ ]:
#Reading the different Excel Sheets containing different Data Columns
#Reading the AHU 2 dat  and dat st pt
df5 = read_excel('BdxData.xlsx',sheet_name='Sheet2')
df5.insert(loc=0, column='Dates', value=to_datetime(df5['Date'],format='%m/%d/%Y %H:%M'))

#Setting Dates as the dataframe index
df5.set_index(['Dates'], drop=True, inplace=True)
df5.drop('Date',axis=1,inplace=True)#Drop original Time column

#Dropping any row with ill conditioned data like ??? etc
for i in df5.columns:
    df5 = df5[~df5[i].apply(lambda x: isinstance(x, str))]

#Ensuring objects are all converted to float
df5 = df5.astype('float')

#Dropiing duplicated time points that may exist in the data
df5 = df5[~df5.index.duplicated()]
df5.index.is_unique #Statutory Check

#Dropping empty rows
df5.dropna(axis=0,how='any',inplace=True)

* Have to collect preheat for ahus 1 and 2 from metasys
* Still need to collect the zone temps
* Or we can collect the entire ahu 1&2 preheat, reheat data from metasys in one go
* The bdx does not have DA_T data
* ALSO HAVE TO CHECKI REMOVE THE SPARSE COLUMNSSO THAT TOO MANY DATA POINTS ARE NOT REMOVED
# THE PHT for AHUs 1 and 2

In [ ]:
#Reading the different Excel Sheets containing different Data Columns
#Reading the AHU 1 and 2 pht temp
df6 = read_excel('AHU12_PH_T_Data.xlsx',sheet_name='Sheet1')
df6.insert(loc=0, column='Dates', value=to_datetime(df6['Date'],format='%m/%d/%Y %H:%M'))

#Setting Dates as the dataframe index
df6.set_index(['Dates'], drop=True, inplace=True)
df6.drop('Date',axis=1,inplace=True)#Drop original Time column

#Dropping any row with ill conditioned data like ??? etc
for i in df6.columns:
    df6 = df6[~df6[i].apply(lambda x: isinstance(x, str))]

#Ensuring objects are all converted to float
df6 = df6.astype('float')

#Dropiing duplicated time points that may exist in the data
df6 = df6[~df6.index.duplicated()]
df6.index.is_unique #Statutory Check

#Dropping empty rows
df6.dropna(axis=0,how='any',inplace=True)

Merge the data frames to create input and output(only df3 energy)
Next step is to collect the zone data and extend all the data to current time point